In [1]:
%load_ext autoreload
%autoreload 2
# %load_ext line_profiler
import h5py
import randomcolor
import numpy as np

from utils import *
from optimization import *
from topological_graph import *

from scipy.spatial import cKDTree
from scipy.interpolate import UnivariateSpline, splev, splprep
from scipy.optimize import minimize
from glob import glob
from tqdm import tqdm
import point_cloud_utils as pcu
import torch
from torch_geometric.nn import fps as tfps
import time
from joblib import Parallel, delayed
from sklearn.model_selection import ParameterGrid 
from sklearn.neighbors import KNeighborsRegressor
from pytorch3d.loss.chamfer import chamfer_distance
from geomloss import SamplesLoss
import copy
from sklearn.metrics import mean_squared_error
import os
import pandas as pd
import pickle

In [2]:
def pred_sampling_points(straight_lines,tcks,closed_tcks):
    pred_sampling = []

    for i in range(len(straight_lines)):
        edge = straight_lines[i]
        num = int(np.linalg.norm(edge[1] - edge[0]) // 0.001)
        if num > 0:
            linspace = np.linspace(0, 1, num)
        #     break
            pred_sampling.append(linspace[:,None] * edge[0] + (1 - linspace)[:,None] * edge[1])
        else:
            pred_sampling.append(edge)

    for i in range(len(tcks)):
        pred_sampling.append(np.array(splev(np.linspace(0,1,5000), tcks[i])).T)

    for i in range(len(closed_tcks)):    
        pred_sampling.append(np.array(splev(np.linspace(0,1,5000), closed_tcks[i])).T)
    return pred_sampling

def curve_length(curve,devision_points_count=200):
    """
    input:
        curve - curve in as in scipy splprep
        devision_points_count - the amount of points on curve
    output:
        length of the curve
    """
    ar = np.array(splev(np.linspace(0,1,devision_points_count), curve)).T
    return np.sum(np.sqrt(np.sum((ar[:-1]-ar[1:])**2,axis=1)))

def metric_calculation(pred_sampling,gt_sampling,blur=0.05):
  
    CD_gt_to_pred = (cKDTree(pred_sampling).query(gt_sampling, 1)[0]**2).mean()
    CD_pred_to_gt = (cKDTree(gt_sampling).query(pred_sampling, 1)[0]**2).mean()
    
    pred_sampling = torch.tensor(pred_sampling)[None]
    gt_sampling = torch.tensor(gt_sampling)[None]
    chamf_l = chamfer_distance(pred_sampling.clone().cuda().detach().float(), gt_sampling.clone().cuda().detach().float())
    
    if pred_sampling[0].shape[0]>10000:
        sub_idx = pcu.downsample_point_cloud_poisson_disk(np.array(pred_sampling[0]), num_samples=10000)
        # Use the indices to get the sample positions and normals
        pred_sampling_small = pred_sampling[:,sub_idx]
    else:
        pred_sampling_small =  pred_sampling
    
    if gt_sampling[0].shape[0]>10000:
        sub_idx = pcu.downsample_point_cloud_poisson_disk(np.array(gt_sampling[0]), num_samples=10000)
        # Use the indices to get the sample positions and normals
        gt_sampling_small = gt_sampling[:,sub_idx]
    else:
        gt_sampling_small = gt_sampling
    sink_ls, en_ls, gaus_ls, lap_ls = loss_calculating(pred_sampling_small,gt_sampling_small,blur=blur)
    return CD_gt_to_pred, CD_pred_to_gt,chamf_l, sink_ls, en_ls, gaus_ls, lap_ls

def write_results(folder_name,straight_lines,tcks,closed_tcks):
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
    with open(folder_name + '/vectorization_line.txt', "wb") as fp:   #Pickling
        pickle.dump(straight_lines, fp)
    with open(folder_name+'/vectorization_open_curves.txt', "wb") as fp:   #Pickling
        pickle.dump(tcks, fp)
    with open(folder_name+'/vectorization_closed_curves.txt', "wb") as fp:   #Pickling
        pickle.dump(closed_tcks, fp)
        
def read_results(folder_name):
    with open(folder_name + '/vectorization_line.txt', "rb") as fp:   # Unpickling
        straight_lines = pickle.load(fp)
    with open(folder_name+'/vectorization_open_curves.txt', "rb") as fp:   #Pickling
        tcks = pickle.load(fp)
    with open(folder_name+'/vectorization_closed_curves.txt', "rb") as fp:   #Pickling
        closed_tcks = pickle.load(fp)
    return straight_lines,tcks,closed_tcks

def loss_calculating(pred_sampling_small,inp_sampling_small,blur=0.05):
    loss = SamplesLoss(loss="sinkhorn",backend = "tensorized",blur=blur)
    sink_ls = loss(pred_sampling_small.cuda(), inp_sampling_small.cuda())

    loss = SamplesLoss(loss="energy",backend = "tensorized",blur=blur)
    en_ls = loss(pred_sampling_small.cuda(), inp_sampling_small.cuda())

    loss = SamplesLoss(loss="gaussian",backend = "tensorized",blur=blur)
    gaus_ls = loss(pred_sampling_small.cuda(), inp_sampling_small.cuda())

    loss = SamplesLoss(loss="laplacian",backend = "tensorized",blur=blur)
    lap_ls = loss(pred_sampling_small.cuda(), inp_sampling_small.cuda())

    return sink_ls, en_ls, gaus_ls, lap_ls
def write_to_file(folder_name, file_name, CD_gt_to_pred, CD_pred_to_gt,chamf_l, sink_ls, en_ls, gaus_ls, lap_ls):
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
    with open(folder_name +'/'+'metrics_'+ file_name + '.txt', 'w') as f:
        f.write('CD_gt_to_pred')
        f.write('\n')
        f.write(str(CD_gt_to_pred.item()))
        f.write('\n')
        f.write('CD_pred_to_gt')
        f.write('\n')
        f.write(str(CD_pred_to_gt.item()))
        f.write('\n')
        f.write('Chamfer loss')
        f.write('\n')
        f.write(str(chamf_l))
        f.write('\n')
        f.write('Energy loss')
        f.write('\n')
        f.write(str(en_ls.item()))
        f.write('\n')
        f.write('Sinkhorn loss')
        f.write('\n')
        f.write(str(sink_ls.item()))
        f.write('\n')
        f.write('Gaussian loss')
        f.write('\n')
        f.write(str(gaus_ls.item()))
        f.write('\n')
        f.write('Laplassian loss')
        f.write('\n')
        f.write(str(lap_ls.item()))
        f.write('\n')

# By one

In [8]:
id_it = 7

In [9]:
path = 'pienet_results/'
_ids = np.sort(glob('pienet_results/*.hdf5'))
# _id = 'abc_0051_00514480_6c33de245ad4c4ff41a3360f_000'
# _id = _ids[id_it].split('__')[0][6:]
_id = _ids[id_it].split('__')[0][15:-5]
_id

'abc_0005_00051370_c304c82e5c884522bb82d7af_000'

In [10]:
len(_ids)

95

In [11]:
### print('Processing ', _id)
# with h5py.File('{path}/{_id}__min.hdf5'.format(path=path, _id=_id), 'r') as f:
with h5py.File('{path}/{_id}.hdf5'.format(path=path, _id=_id), 'r') as f:
    print(list(f.keys()))
    point_cloud = f['point_cloud'][:]
    open_curv = f['open_curves'][:]
    close_curv = f['closed_curves'][:]

['closed_curves', 'open_curves', 'point_cloud']


In [12]:
pred_sampling = np.concatenate((np.array(close_curv).reshape(-1,3),np.array(open_curv).reshape(-1,3)),axis=0)

In [13]:
pred_sampling.shape

(0, 3)

In [74]:
# save this also
# if not NO_GT:
plot = k3d.plot()

cmap=k3d.colormaps.matplotlib_color_maps.plasma_r

k3d_points = k3d.points(pred_sampling, point_size=0.03, opacity=0.1, shader='3d', name='sharp_points', color=0xff0000)
plot += k3d_points

plot.display()

Output()

# Writing metrics and curves

In [77]:
import os
NO_GT=False
# load GT curves as segments
gt_edges = []
if os.path.exists('parametric_labeling/{file}__parametric.txt'.format(file=_id[9:])):
    with open('parametric_labeling/{file}__parametric.txt'.format(file=_id[9:])) as f:
        for line in f:
            curve_id, curve_type, edges = line.split(' ', maxsplit=2)
            gt_edges.append(np.array([float(v) for v in edges.split()]).reshape((-1, 2, 3)))
else:
    NO_GT = True
    print('No GT!')

In [78]:
# sample GT edges
if not NO_GT:
    gt_sampling = []
    for edge in np.concatenate(gt_edges):
        num = int(np.linalg.norm(edge[1] - edge[0]) // 0.001)
        if num > 0:
            linspace = np.linspace(0, 1, num)
        #     break
            gt_sampling.append(linspace[:,None] * edge[0] + (1 - linspace)[:,None] * edge[1])
        else:
            gt_sampling.append(edge)
    gt_sampling = np.concatenate(gt_sampling)

In [79]:
if not NO_GT:
    CD_gt_to_pred, CD_pred_to_gt,chamf_l, sink_ls, en_ls, gaus_ls, lap_ls  = metric_calculation(pred_sampling,gt_sampling)
    _, _,chamf_l, sink_ls_bl02, en_ls_bl02, gaus_ls_bl02, lap_ls_bl02  = metric_calculation(pred_sampling,gt_sampling)

In [80]:
folder_name = 'pienet_results'
meta_exp_name = folder_name
if not NO_GT:
    write_to_file(folder_name+'/metrics/',str(_id),CD_gt_to_pred, CD_pred_to_gt,chamf_l, sink_ls, en_ls, gaus_ls, lap_ls)
    df = pd.DataFrame([[str(_id),CD_gt_to_pred.item(),CD_pred_to_gt.item(),chamf_l[0].item(),
                        en_ls.item(),sink_ls.item(),
                        gaus_ls.item(),lap_ls.item(), en_ls_bl02.item(),sink_ls_bl02.item(),
                        gaus_ls_bl02.item(),lap_ls_bl02.item()]],
                      index=None,  columns=['Name','CD_gt_to_pred','CD_pred_to_gt','Chamfer_loss',
                                            'Energy_loss','Sinkhorn_loss','Gaussian_loss'
                                            ,'Laplassian_loss','Energy_loss_blure02','Sinkhorn_loss_blure02','Gaussian_loss_blure02'
                                            ,'Laplassian_loss_blure02'])
    if not os.path.exists(meta_exp_name+'/metrics/'):
        os.makedirs(meta_exp_name+'/metrics/')
    
    if not os.path.exists(meta_exp_name+'/metrics/'+meta_exp_name + '_final_metrics.csv'):
        df.to_csv(meta_exp_name+'/metrics/'+meta_exp_name + '_final_metrics.csv', index = False,sep=';')
    else:
        df.to_csv(meta_exp_name+'/metrics/'+meta_exp_name + '_final_metrics.csv', mode='a', header=None, index = False,sep=';')

In [81]:
# save this also
if not NO_GT:
    plot = k3d.plot()

    cmap=k3d.colormaps.matplotlib_color_maps.plasma_r

    k3d_points = k3d.points(pred_sampling, point_size=0.03, opacity=0.1, shader='3d', name='sharp_points', color=0xff0000)
    plot += k3d_points

    k3d_points = k3d.points(gt_sampling, point_size=0.03, opacity=0.1, shader='3d', name='sharp_points')
    plot += k3d_points

    plot.display()
    with open(folder_name+'/plot_with_GT/'+str(_id)+'.html', 'w') as f:
        f.write(plot.get_snapshot())

Output()

# For whole batch

In [17]:
folder_name = 'pienet_results'
for id_it in range(0,95):
    path = 'pienet_results/'
    _ids = np.sort(glob('pienet_results/*.hdf5'))
    _id = _ids[id_it].split('__')[0][15:-5]
    print(id_it,_id)
    with h5py.File('{path}/{_id}.hdf5'.format(path=path, _id=_id), 'r') as f:
        point_cloud = f['point_cloud'][:]
        open_curv = f['open_curves'][:]
        close_curv = f['closed_curves'][:]
    pred_sampling = np.concatenate((np.array(close_curv).reshape(-1,3),np.array(open_curv).reshape(-1,3)),axis=0)
    if pred_sampling.size == 0:
        print('empty_array:', id_it,_id)
        continue
    NO_GT=False
    # load GT curves as segments
    gt_edges = []
    if os.path.exists('parametric_labeling/{file}__parametric.txt'.format(file=_id[9:])):
        with open('parametric_labeling/{file}__parametric.txt'.format(file=_id[9:])) as f:
            for line in f:
                curve_id, curve_type, edges = line.split(' ', maxsplit=2)
                gt_edges.append(np.array([float(v) for v in edges.split()]).reshape((-1, 2, 3)))
    else:
        NO_GT = True
        print('No GT!')
    
    # sample GT edges
    if not NO_GT:
        gt_sampling = []
        for edge in np.concatenate(gt_edges):
            num = int(np.linalg.norm(edge[1] - edge[0]) // 0.001)
            if num > 0:
                linspace = np.linspace(0, 1, num)
            #     break
                gt_sampling.append(linspace[:,None] * edge[0] + (1 - linspace)[:,None] * edge[1])
            else:
                gt_sampling.append(edge)
        gt_sampling = np.concatenate(gt_sampling)
    if not NO_GT:
        CD_gt_to_pred, CD_pred_to_gt,chamf_l, sink_ls, en_ls, gaus_ls, lap_ls  = metric_calculation(pred_sampling,gt_sampling)
        _, _,chamf_l, sink_ls_bl02, en_ls_bl02, gaus_ls_bl02, lap_ls_bl02  = metric_calculation(pred_sampling,gt_sampling)
    
    meta_exp_name = folder_name
    if not NO_GT:
        write_to_file(folder_name+'/metrics/',str(_id),CD_gt_to_pred, CD_pred_to_gt,chamf_l, sink_ls, en_ls, gaus_ls, lap_ls)
        df = pd.DataFrame([[str(_id),CD_gt_to_pred.item(),CD_pred_to_gt.item(),chamf_l[0].item(),
                            en_ls.item(),sink_ls.item(),
                            gaus_ls.item(),lap_ls.item(), en_ls_bl02.item(),sink_ls_bl02.item(),
                            gaus_ls_bl02.item(),lap_ls_bl02.item()]],
                          index=None,  columns=['Name','CD_gt_to_pred','CD_pred_to_gt','Chamfer_loss',
                                                'Energy_loss','Sinkhorn_loss','Gaussian_loss'
                                                ,'Laplassian_loss','Energy_loss_blure02','Sinkhorn_loss_blure02','Gaussian_loss_blure02'
                                                ,'Laplassian_loss_blure02'])
        if not os.path.exists(meta_exp_name+'/metrics/'):
            os.makedirs(meta_exp_name+'/metrics/')

        if not os.path.exists(meta_exp_name+'/metrics/'+meta_exp_name + '_final_metrics.csv'):
            df.to_csv(meta_exp_name+'/metrics/'+meta_exp_name + '_final_metrics.csv', index = False,sep=';')
        else:
            df.to_csv(meta_exp_name+'/metrics/'+meta_exp_name + '_final_metrics.csv', mode='a', header=None, index = False,sep=';')
        # save this also
    
    if not NO_GT:
        plot = k3d.plot()

        cmap=k3d.colormaps.matplotlib_color_maps.plasma_r

        k3d_points = k3d.points(pred_sampling, point_size=0.03, opacity=0.1, shader='3d', name='sharp_points', color=0xff0000)
        plot += k3d_points

        k3d_points = k3d.points(gt_sampling, point_size=0.03, opacity=0.1, shader='3d', name='sharp_points')
        plot += k3d_points

#         plot.display()
        with open(folder_name+'/plot_with_GT/'+str(_id)+'.html', 'w') as f:
            f.write(plot.get_snapshot())

0 abc_0000_00000006_d4fe04f0f5f84b52bd4f10e4_001
1 abc_0000_00000068_767e4372b5f94a88a7a17d90_009
2 abc_0000_00000077_767e4372b5f94a88a7a17d90_018
3 abc_0000_00000162_4dcda9fab6dd46cf98fb194e_000
4 abc_0004_00041400_68770188fbbf4d658868ae7a_000
5 abc_0004_00046950_06d3284aaeba4d5e8eb6d441_000
6 abc_0005_00051130_fc39aebf81cd429e91825d67_000
7 abc_0005_00051370_c304c82e5c884522bb82d7af_000
empty_array: 7 abc_0005_00051370_c304c82e5c884522bb82d7af_000
8 abc_0005_00057780_da518f0dbee046f287c2acf5_000
9 abc_0050_00500041_5aa40dcd43fa0b14df9bdcf8_010
10 abc_0050_00500149_54930d6f7740b03347d89a56_000
11 abc_0050_00500348_fae0ecd8b3dc068d39f0d09c_000
12 abc_0050_00500683_511f7debb63f164003339dec_000
13 abc_0050_00500912_ded12ea6d13dadf860aaa4cb_000
No GT!
14 abc_0050_00507625_b39680dc8dc650ec01956bbe_011
No GT!
15 abc_0050_00508845_266ea1913e2d35805fb70e9d_001
16 abc_0050_00509172_c683c0212890fcb20340e86c_000
17 abc_0051_00510038_1509fe78c7a4729189287091_000
18 abc_0051_00510073_951e25d2ded40